## Criação do dataset de features para musicas boas

In [54]:
!pip install ipython-autotime
%load_ext autotime
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 6.99 s (started: 2022-06-19 19:11:54 +00:00)


In [55]:
import pandas as pd
import numpy as np
import glob,os

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.11 s (started: 2022-06-19 19:12:01 +00:00)


In [56]:
# Usuário: fijita8647@karavic.com
# Senha: INF10322803
cid ="17fd5fc78c79479fb19bd36652021b46" 
secret = "85ee26c27c4f4601a3c6aef838a80e89"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

time: 4.9 ms (started: 2022-06-19 19:12:03 +00:00)


In [57]:
# lista_uris_playlists_ = ["spotify:playlist:4hJvhulqLezauCnTRb2yPq","spotify:playlist:7lvHsa9sDh1g1Qixh6oSdP",
#                               "spotify:playlist:1fsmqAwNtzvPUIYjvQ79Xv","spotify:playlist:37i9dQZF1DWWH0izG4erma",
#                               "spotify:playlist:37i9dQZF1DX52ln8eMkne9","spotify:playlist:2lTR2BqF6bUQN1rIRpQF47",
#                               "spotify:playlist:4VRQJWxn3HqJuuVEqfnuev",
#                               "spotify:playlist:37i9dQZF1DX504r1DvyvxG","spotify:playlist:7gUZfcI8pUFTyMsXkrzPb1",
#                               "spotify:playlist:29DClu3jqLIbyy3FydsSaI","spotify:playlist:4lQ2XTitOc5GdNWgexMWHy",
#                               "spotify:playlist:4HISklhCkavPUTxeuGfgbA","spotify:playlist:7HKWqYXzLDGcgnHbfv4N2F",
#                               "spotify:playlist:2vXoumS4E9fMFUktyWAG1F","spotify:playlist:4lx98oJ3fSdgjolFJ1CF7F",
#                               "spotify:playlist:5tkdpecyoo1o3WDfE74wxC","spotify:playlist:6tq2sQwVvaofZfR34Eofjs",
#                               "spotify:playlist:3RhbGg2TKlRbo5lfEtTw0F","spotify:playlist:1GnlE9hxKHyte0f5wqfUdB",
#                               "spotify:playlist:3vDe8D64ytZRKXt0AsJT0B","spotify:playlist:3SB2pblL5GhBmyw6ucs0bl",
#                               "spotify:playlist:3f99pCj7qQN5L0Srbl8ea7","spotify:playlist:0S7g5dnTxitSdvg9vhw147"]#,"spotify:playlist:4ydUfAuEuxYjLl1hlPYbJo","spotify:playlist:7CjhiKrsLt2M0xgddMgqDH"]



time: 11.4 ms (started: 2022-06-19 19:12:03 +00:00)


In [58]:
# def get_num_tracks(playlist_id):
#   playlist = sp.playlist_tracks(playlist_id=playlist_id)
#   total_tracks_na_playlist = playlist["total"]
#   return total_tracks_na_playlist

# def get_tracks_from_playlist(playlist_id,total_tracks):
#   print(playlist_id)
#   num_consultas = total_tracks//50
#   offset = 0

#   lista_uris_tracks = []
#   for i in range(num_consultas+1):
#     playlist = sp.playlist_tracks(playlist_id=playlist_id,offset=offset)
#     offset+=50
#     if(playlist['items']!= []):
#       for item in playlist['items']:
#         # print(type(item['track']['uri']))
#         if type(item['track']['uri']) == type("string"):
#           # print(item['track']['uri'])
#           lista_uris_tracks.append(item['track']['uri'])
#     print(offset) 
      

#   return lista_uris_tracks

time: 3.95 ms (started: 2022-06-19 19:12:03 +00:00)


In [59]:
def pega_uri_charts(path):

  #Pega todos os arquivos do diretorio
  arquivos = glob.glob(os.path.join(path,"*.csv"))
  dados = pd.DataFrame()

  for file in arquivos:
      dados_temp = pd.read_csv(file)
      #Coloca o nome certo nas colunas
      dados_temp.columns = dados_temp.iloc[0]
      dados_temp = dados_temp.drop(axis=0,index=0)
      #Vai concatenando no dataet final
      dados = pd.concat([dados,dados_temp])

  #Preciso apenas do URI, Streams, Nome e Artista
  dados = dados[["URL","Streams","Track Name","Artist"]]

  #Para analise de anomalias nao preciso da informacao de quantas vezes a musica apareceu no top 200, entao removo duplicatas
  dados = dados.drop_duplicates(keep="first")

  #Reset no index, ja que removi linhas
  dados = dados.reset_index(drop=True)
  
  return dados

time: 14.6 ms (started: 2022-06-19 19:12:03 +00:00)


In [60]:
# path_global = "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/global"
# lista_uris_musicas_boas = pega_uri_charts(path_global)
# lista_uris_musicas_boas = lista_uris_musicas_boas.URL

path_ja_feito = "/content/drive/MyDrive/INF1032 - Spotify/Dados/Matheus/MusicasAnomalas/top200_global_consolidado.csv"
lista_uris_musicas_boas = pd.read_csv(path_ja_feito)
lista_uris_musicas_boas = lista_uris_musicas_boas.uri

time: 62.2 ms (started: 2022-06-19 19:12:03 +00:00)


In [72]:
def get_audio_features_from_tracks(lista_uris_musicas_):

  max_tracks = 50
  df_features =[]
  for i in range(0,len(lista_uris_musicas_),max_tracks):
    lista_features = sp.audio_features(lista_uris_musicas_[i:i+max_tracks])
    for features in lista_features:
      # print(features)
      df_features.append(features.values())
                            
  df_features = pd.DataFrame(df_features)
  df_features.columns = features.keys()
  df_features = df_features.reset_index(drop=True)  

  return df_features

def get_outras_features_from_tracks(lista_uris_musicas_):

  max_tracks = 50
  df_features = pd.DataFrame(columns=["nome","data_lancamento","Popularidade Musica","Artista"])
  for i in range(0,len(lista_uris_musicas_),max_tracks):
    track_info = sp.tracks(lista_uris_musicas_[i:i+max_tracks])

    for track in track_info['tracks']:
      data_lancamento = track['album']['release_date']
      nome_musica = track['name']
      popularidade_musica = track["popularity"]
      nome_artista = track['artists'][0]['name']
      uri_artista = track["artists"][0]['uri']    

      df_features.loc[len(df_features.index)] = [nome_musica,data_lancamento,popularidade_musica,nome_artista]
  
  df_features = df_features.reset_index(drop=True)
  return df_features

def get_artist_features(lista_uris_musicas_):
  max_tracks = 50
  df_features = pd.DataFrame(columns=["Popularidade Artista","Seguidores","Estilos"])
  lista_uris_artista = []
  for i in range(0,len(lista_uris_musicas_),max_tracks):
    track_info = sp.tracks(lista_uris_musicas_[i:i+max_tracks])

    for track in track_info['tracks']:
      uri_artista = track['artists'][0]['uri']
      lista_uris_artista.append(uri_artista)
    
  for i in range(0,len(lista_uris_artista),max_tracks):
    artist_info = sp.artists(lista_uris_artista[i:i+max_tracks])
    
    for artist in artist_info['artists']:
      # print(artist.keys())
      pop_artista = artist['popularity']
      folow_artista = artist['followers']['total']
      estilos_artista = artist['genres']    

      df_features.loc[len(df_features.index)] = [pop_artista,folow_artista,estilos_artista]

  df_features = df_features.reset_index(drop=True)
  return df_features

time: 91.5 ms (started: 2022-06-19 19:15:40 +00:00)


## Audio Features

In [62]:
df_audio_features_musicas_boas = get_audio_features_from_tracks(lista_uris_musicas_boas)

time: 5.33 s (started: 2022-06-19 19:12:03 +00:00)


In [63]:
df_audio_features_musicas_boas

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,0.905,0.563,8,-6.135,1,0.1020,0.02540,0.000010,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870,audio_features,02MWAaffLxlfxAUY7c5dvx,spotify:track:02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,https://api.spotify.com/v1/audio-analysis/02MW...,238805,4
3,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,0.478,169.928,audio_features,5PjdY0CKGZdEuoNab3yDmX,spotify:track:5PjdY0CKGZdEuoNab3yDmX,https://api.spotify.com/v1/tracks/5PjdY0CKGZdE...,https://api.spotify.com/v1/audio-analysis/5Pjd...,141806,4
4,0.756,0.697,8,-6.377,1,0.0401,0.18200,0.000000,0.3330,0.956,94.996,audio_features,2DB4DdfCFMw1iaR6JaR03a,spotify:track:2DB4DdfCFMw1iaR6JaR03a,https://api.spotify.com/v1/tracks/2DB4DdfCFMw1...,https://api.spotify.com/v1/audio-analysis/2DB4...,206071,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,0.714,0.442,6,-5.909,1,0.0605,0.74200,0.000000,0.1140,0.546,108.039,audio_features,1jIMjbzcGCcCQn5iMu9CUc,spotify:track:1jIMjbzcGCcCQn5iMu9CUc,https://api.spotify.com/v1/tracks/1jIMjbzcGCcC...,https://api.spotify.com/v1/audio-analysis/1jIM...,146855,4
5152,0.601,0.713,4,-3.758,0,0.0449,0.02820,0.000000,0.1580,0.464,122.872,audio_features,1WCEAGGRD066z2Q89ObXTq,spotify:track:1WCEAGGRD066z2Q89ObXTq,https://api.spotify.com/v1/tracks/1WCEAGGRD066...,https://api.spotify.com/v1/audio-analysis/1WCE...,169756,4
5153,0.578,0.431,2,-7.034,1,0.0269,0.46900,0.000000,0.1370,0.210,116.979,audio_features,1TUuhV75FeOF6UObRsABo2,spotify:track:1TUuhV75FeOF6UObRsABo2,https://api.spotify.com/v1/tracks/1TUuhV75FeOF...,https://api.spotify.com/v1/audio-analysis/1TUu...,226975,3
5154,0.340,0.684,11,-6.306,1,0.0440,0.00979,0.000000,0.3480,0.556,112.969,audio_features,7AYP21Q4qnxw2WxETEvSRb,spotify:track:7AYP21Q4qnxw2WxETEvSRb,https://api.spotify.com/v1/tracks/7AYP21Q4qnxw...,https://api.spotify.com/v1/audio-analysis/7AYP...,248819,4


time: 49.7 ms (started: 2022-06-19 19:12:08 +00:00)


## Demais Features

In [64]:
df_outras_features_musicas_boas = get_outras_features_from_tracks(lista_uris_musicas_boas)

time: 30.3 s (started: 2022-06-19 19:12:09 +00:00)


In [65]:
df_outras_features_musicas_boas

,nome,data_lancamento,Popularidade Musica,Artista
0,As It Was,2022-03-31,100,Harry Styles
1,First Class,2022-04-08,94,Jack Harlow
2,Heat Waves,2020-08-06,90,Glass Animals
3,STAY (with Justin Bieber),2021-07-23,88,The Kid LAROI
4,Bam Bam (feat. Ed Sheeran),2022-04-07,85,Camila Cabello
...,...,...,...,...
5151,That's Hilarious,2022-04-08,81,Charlie Puth
5152,Crazy What Love Can Do,2022-04-08,86,David Guetta
5153,Don't Think Jesus,2022-04-15,79,Morgan Wallen
5154,Memories,2022-04-15,81,Conan Gray


time: 16.1 ms (started: 2022-06-19 19:12:39 +00:00)


In [66]:
def add_features_data_lancamento(df):

  df["data_lancamento"] = pd.to_datetime(df["data_lancamento"],errors = 'coerce')
  df["ano_lancamento"] = df["data_lancamento"].dt.year
  df["mes_lancamento"] = df["data_lancamento"].dt.month
  df["dia_semana_lancamento"] = df["data_lancamento"].dt.weekday

  return df

time: 3.95 ms (started: 2022-06-19 19:12:39 +00:00)


In [67]:
df_outras_features_musicas_boas = add_features_data_lancamento(df_outras_features_musicas_boas)
df_outras_features_musicas_boas

,nome,data_lancamento,Popularidade Musica,Artista,ano_lancamento,mes_lancamento,dia_semana_lancamento
0,As It Was,2022-03-31,100,Harry Styles,2022.0,3.0,3.0
1,First Class,2022-04-08,94,Jack Harlow,2022.0,4.0,4.0
2,Heat Waves,2020-08-06,90,Glass Animals,2020.0,8.0,3.0
3,STAY (with Justin Bieber),2021-07-23,88,The Kid LAROI,2021.0,7.0,4.0
4,Bam Bam (feat. Ed Sheeran),2022-04-07,85,Camila Cabello,2022.0,4.0,3.0
...,...,...,...,...,...,...,...
5151,That's Hilarious,2022-04-08,81,Charlie Puth,2022.0,4.0,4.0
5152,Crazy What Love Can Do,2022-04-08,86,David Guetta,2022.0,4.0,4.0
5153,Don't Think Jesus,2022-04-15,79,Morgan Wallen,2022.0,4.0,4.0
5154,Memories,2022-04-15,81,Conan Gray,2022.0,4.0,4.0


time: 35.2 ms (started: 2022-06-19 19:12:39 +00:00)


In [73]:
df_features_artistas = get_artist_features(lista_uris_musicas_boas)

time: 31.7 s (started: 2022-06-19 19:15:45 +00:00)


In [74]:
df_features_artistas

,Popularidade Artista,Seguidores,Estilos
0,94,21444145,[pop]
1,86,2247792,"[deep underground hip hop, kentucky hip hop, rap]"
2,80,2960684,"[gauze pop, indietronica, shiver pop]"
3,83,3778109,[australian hip hop]
4,83,27026106,"[dance pop, pop, post-teen pop]"
...,...,...,...
5151,80,16374282,"[dance pop, pop, viral pop]"
5152,85,24379078,"[big room, dance pop, edm, pop, pop dance, pop..."
5153,83,3515158,[contemporary country]
5154,78,5768997,"[bedroom pop, pop]"


time: 20.3 ms (started: 2022-06-19 19:16:17 +00:00)


## Juntando todos os datasets

In [75]:
df_musicas_ = pd.concat([df_audio_features_musicas_boas,df_outras_features_musicas_boas,df_features_artistas],axis=1)
path = '/content/drive/My Drive/INF1032 - Spotify/Dados/Consolidados/'
df_musicas_.to_csv(path+"musicas_boas.csv")
df_musicas_

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,nome,data_lancamento,Popularidade Musica,Artista,ano_lancamento,mes_lancamento,dia_semana_lancamento,Popularidade Artista,Seguidores,Estilos
0,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,...,As It Was,2022-03-31,100,Harry Styles,2022.0,3.0,3.0,94,21444145,[pop]
1,0.905,0.563,8,-6.135,1,0.1020,0.02540,0.000010,0.1130,0.324,...,First Class,2022-04-08,94,Jack Harlow,2022.0,4.0,4.0,86,2247792,"[deep underground hip hop, kentucky hip hop, rap]"
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,...,Heat Waves,2020-08-06,90,Glass Animals,2020.0,8.0,3.0,80,2960684,"[gauze pop, indietronica, shiver pop]"
3,0.591,0.764,1,-5.484,1,0.0483,0.03830,0.000000,0.1030,0.478,...,STAY (with Justin Bieber),2021-07-23,88,The Kid LAROI,2021.0,7.0,4.0,83,3778109,[australian hip hop]
4,0.756,0.697,8,-6.377,1,0.0401,0.18200,0.000000,0.3330,0.956,...,Bam Bam (feat. Ed Sheeran),2022-04-07,85,Camila Cabello,2022.0,4.0,3.0,83,27026106,"[dance pop, pop, post-teen pop]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151,0.714,0.442,6,-5.909,1,0.0605,0.74200,0.000000,0.1140,0.546,...,That's Hilarious,2022-04-08,81,Charlie Puth,2022.0,4.0,4.0,80,16374282,"[dance pop, pop, viral pop]"
5152,0.601,0.713,4,-3.758,0,0.0449,0.02820,0.000000,0.1580,0.464,...,Crazy What Love Can Do,2022-04-08,86,David Guetta,2022.0,4.0,4.0,85,24379078,"[big room, dance pop, edm, pop, pop dance, pop..."
5153,0.578,0.431,2,-7.034,1,0.0269,0.46900,0.000000,0.1370,0.210,...,Don't Think Jesus,2022-04-15,79,Morgan Wallen,2022.0,4.0,4.0,83,3515158,[contemporary country]
5154,0.340,0.684,11,-6.306,1,0.0440,0.00979,0.000000,0.3480,0.556,...,Memories,2022-04-15,81,Conan Gray,2022.0,4.0,4.0,78,5768997,"[bedroom pop, pop]"


time: 232 ms (started: 2022-06-19 19:16:17 +00:00)


# Pegando a partir do dataset global final

In [ ]:
# path_consolidados = '/content/drive/MyDrive/INF1032 - Spotify/Dados/Consolidados/'
# df_global_final = pd.read_csv(path_consolidados+"global_final.csv")

In [ ]:
# df_global_final.to_csv(path_consolidados+"musicas_boas")